# セットアップ

In [1]:
from datetime import date,timedelta
import pandas as pd
import numpy as np
import pickle
with open("all_of_dic.pkl", "rb") as file:
    all_df_dic = pickle.load(file)

In [115]:
df_accounting = pd.DataFrame(index = all_df_dic["有価証券報告書"].keys(), columns = ["company_name", "accounting"])

def get_unique_element1(df, col_name):
    df_focus = df[df["tag"] == col_name].copy()
    if len(df_focus) == 1:
        return df_focus["値"].values[0]
    elif len(df_focus) == 0:
        return np.nan
    else:
        print("Error: 値がユニークでない" + " " + col_name)
        return "Error: 値がユニークでない：" + len(df_focus)
    
def get_unique_element2(df, col_name):
    df_focus = df[df["tag"] == col_name].copy()
    val_return = np.nan
    if "CurrentYearInstant" in df_focus["context"].values:
        val_return = df_focus[df_focus["context"] == "CurrentYearInstant"]["値"].values[0]
    elif "CurrentYearInstant_NonConsolidatedMember" in df_focus["context"].values:
        val_return = df_focus[df_focus["context"] == "CurrentYearInstant_NonConsolidatedMember"]["値"].values[0]
    return val_return

for idx, code in enumerate(all_df_dic["有価証券報告書"].keys()):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    company_name = get_unique_element1(df_idx, "FilerNameInJapaneseDEI")
    accounting = get_unique_element1(df_idx, "AccountingStandardsDEI")
    
    df_accounting.at[code, "company_name"] = company_name
    df_accounting.at[code, "accounting"] = accounting

In [53]:
col_JGAAP = ["ShareholdersEquity", "NetAssets", "SubscriptionRightsToShares", "NonControllingInterests"]
df_JGAAP = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "Japan GAAP"].index, columns = col_JGAAP)

col_IFRS = ["EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults"]
df_IFRS = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "IFRS"].index, columns = col_IFRS)

col_USGAAP = ["EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults"]
df_USGAAP = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "US GAAP"].index, columns = col_USGAAP)

In [116]:
for idx, code in enumerate(all_df_dic["有価証券報告書"].keys()):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    accounting = get_unique_element1(df_idx, "AccountingStandardsDEI")
    if accounting == "Japan GAAP":
        for col in col_JGAAP:
            element = get_unique_element2(df_idx, col)
            df_JGAAP.at[code, col] = element
    
    if accounting == "IFRS":
        for col in col_IFRS:
            element = get_unique_element2(df_idx, col)
            df_IFRS.at[code, col] = element
    
    if accounting == "US GAAP":
        for col in col_USGAAP:
            element = get_unique_element2(df_idx, col)
            df_USGAAP.at[code, col] = element

In [120]:
df_JGAAP.isnull().sum()

ShareholdersEquity               0
NetAssets                        0
SubscriptionRightsToShares    2436
NonControllingInterests       1770
dtype: int64

In [121]:
df_IFRS.isnull().sum()

EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults    9
dtype: int64

In [122]:
df_USGAAP.isnull().sum()

EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults    5
dtype: int64

In [129]:
df_JGAAP.fillna({"SubscriptionRightsToShares":0.0, "NonControllingInterests":0.0}, inplace = True)
df_JGAAP["EQUITY"] = df_JGAAP["NetAssets"] - df_JGAAP["SubscriptionRightsToShares"] - df_JGAAP["NonControllingInterests"]
df_IFRS.rename(columns = {"EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults": "EQUITY"}, inplace = True)
df_USGAAP.rename(columns = {"EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults": "EQUITY"}, inplace = True)

In [133]:
df_EQUITY = pd.concat([df_JGAAP, df_IFRS, df_USGAAP], join = "inner")
df_EQUITY.to_excel("equity_20220130.xlsx")

# もう少しうまくデータを取り出せるかも

In [46]:
df_JGAAP.isnull().sum()
df_JGAAP["Equity"] = df_JGAAP["NetAssets"].values - df_JGAAP["SubscriptionRightsToShares"].values - df_JGAAP["NonControllingInterests"].values
df_JGAAP

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [76]:
df_USGAAP["EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults"].str.contains("Error")

AttributeError: Can only use .str accessor with string values!

In [73]:
df_JGAAP

,ShareholdersEquity,NetAssets,SubscriptionRightsToShares,NonControllingInterests
E03006,53798000000,53765000000,NaN,69000000
E03531,467615000000,490006000000,482000000,-4541000000
E03733,13768545000,13792158000,9529000,7345000
E34704,2223890000,2223890000,NaN,NaN
E03439,NaN,NaN,NaN,NaN
...,...,...,...,...
E05328,4214097000,4159290000,141954000,12497000
E02932,6297295000,6442031000,NaN,29396000
E03460,NaN,NaN,NaN,NaN
E05725,55651000000,55507000000,3000000,70000000


In [75]:
df_IFRS.isnull().sum()

EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults    9
dtype: int64

In [66]:
code = "E03439"
df_idx = all_df_dic["有価証券報告書"][code]
df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
accounting = get_unique_element(df_idx, "AccountingStandardsDEI")
df_idx = df_idx[df_idx["context"] == "CurrentYearInstant"]

col = col_JGAAP[2]
df_idx_col = df_idx[df_idx["tag"] == col]
df_idx_col


,version,提出日,提出回数,報告対象期間期末日,追番,第N期,名前空間接頭辞,tag,id,context,開始日,終了日,期末日,連結,値


In [59]:
print(accounting)

Japan GAAP


In [77]:
def get_unique_element(df, col_name):
    df_focus = df[df["tag"] == col_name].copy()
    if len(df_focus) == 1:
        return df_focus["値"].values[0]
    elif len(df_focus) == 0:
        return np.nan
    else:
        print("Error: 値がユニークでない" + " " + col_name)
        return "Error: 値がユニークでない：" + len(df_focus)

In [31]:
df_accounting["accounting"].value_counts().to_frame()

,accounting
Japan GAAP,3469
IFRS,228
US GAAP,11


In [33]:
df_accounting[df_accounting["accounting"] == "Japan GAAP"]

,company_name,accounting
E03006,あいホールディングス株式会社,Japan GAAP
E03531,株式会社あおぞら銀行,Japan GAAP
E03733,株式会社あかつき本社,Japan GAAP
E34704,株式会社あさくま,Japan GAAP
E03439,株式会社あさひ,Japan GAAP
...,...,...
E05328,株式会社YE DIGITAL,Japan GAAP
E02932,ＹＫＴ株式会社,Japan GAAP
E03460,株式会社ＺＯＡ,Japan GAAP
E05725,株式会社ZOZO,Japan GAAP


In [34]:
df_accounting[df_accounting["accounting"] == "IFRS"]

,company_name,accounting
E35495,株式会社きずなホールディングス,IFRS
E30047,株式会社じげん,IFRS
E03515,株式会社すかいらーくホールディングス,IFRS
E00981,そーせいグループ株式会社,IFRS
E01593,株式会社アイシン,IFRS
...,...,...
E04830,ＳＣＳＫ株式会社,IFRS
E02152,株式会社ＳＵＢＡＲＵ,IFRS
E01678,ＴＨＫ株式会社,IFRS
E02889,ＶＴホールディングス株式会社,IFRS


In [32]:
df_accounting[df_accounting["accounting"] == "US GAAP"]

,company_name,accounting
E01755,オムロン株式会社,US GAAP
E04762,オリックス株式会社,US GAAP
E02274,キヤノン株式会社,US GAAP
E01777,ソニーグループ株式会社,US GAAP
E00590,株式会社ワコールホールディングス,US GAAP
E00988,富士フイルムホールディングス株式会社,US GAAP
E01532,株式会社小松製作所,US GAAP
E01914,株式会社村田製作所,US GAAP
E01738,株式会社 東芝,US GAAP
E03752,野村ホールディングス株式会社,US GAAP


In [79]:
code = "E03439"

df_idx = all_df_dic["有価証券報告書"][code]
df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]

df_idx.to_excel("temp_" + code + ".xlsx")


トヨタ(IFRS)
親会社の所有者に帰属する持分合計
EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults
CurrentYearInstant
資産合計
TotalAssetsIFRSSummaryOfBusinessResults
営業収益合計
OperatingRevenuesIFRSKeyFinancialData
当期利益の帰属(親会社の所有者)
ProfitLossAttributableToOwnersOfParentIFRSSummaryOfBusinessResults

ZOZO(Japan GAAP)
株主資本合計
ShareholdersEquity
純資産合計
NetAssets
新株予約券
SubscriptionRightsToShares
被支配株主持ち分
NonControllingInterests

In [ ]:
ソニーグループ株式会社(US GAAP)
資本合計
EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults



# Japan GAAP

In [106]:
def get_unique_element(df, col_name):
    df_focus = df[df["tag"] == col_name]
    val_return = np.nan
    if "CurrentYearInstant" in df_focus["context"].values:
        val_return = df_focus[df_focus["context"] == "CurrentYearInstant"]["値"].values[0]
    elif "CurrentYearInstant_NonConsolidatedMember" in df_focus["context"].values:
        val_return = df_focus[df_focus["context"] == "CurrentYearInstant_NonConsolidatedMember"]["値"].values[0]
    return val_return

col_JGAAP = ["ShareholdersEquity", "NetAssets", "SubscriptionRightsToShares", "NonControllingInterests"]
df_JGAAP = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "Japan GAAP"].index, columns = col_JGAAP)

for idx, code in enumerate(df_JGAAP.index):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    for col in col_JGAAP:
        element = get_unique_element(df_idx, col)
        df_JGAAP.at[code, col] = element

In [108]:
df_JGAAP.isnull().sum()

ShareholdersEquity               0
NetAssets                        0
SubscriptionRightsToShares    2436
NonControllingInterests       1770
dtype: int64

In [101]:
df_num_elem = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "Japan GAAP"].index, columns = col_JGAAP)
dict_context = dict(zip(col_JGAAP, [[] for _ in range(len(col_JGAAP))]))

for idx, code in enumerate(df_num_elem.index):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    df_idx = df_idx[df_idx["context"].str.contains("CurrentYearInstant")]
    for col in col_JGAAP:
        context_list = df_idx[df_idx["tag"] == col]["context"].values
        df_num_elem.at[code, col] = len(df_idx[df_idx["tag"] == col])
        if col == "NetAssets":
            df_idx_netassets = df_idx[(df_idx["context"] == "CurrentYearInstant")|(df_idx["context"] == "CurrentYearInstant_NonConsolidatedMember")]
            df_num_elem.at[code, col] = len(df_idx_netassets[df_idx_netassets["tag"] == col])
        if len(context_list) != 0:
            dict_context[col].extend(context_list)

# IFRS

In [111]:
df_num_elem = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "IFRS"].index, columns = col_IFRS)
dict_context = dict(zip(col_IFRS, [[] for _ in range(len(col_IFRS))]))

for idx, code in enumerate(df_num_elem.index):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    df_idx = df_idx[df_idx["context"].str.contains("CurrentYearInstant")]
    for col in col_IFRS:
        context_list = df_idx[df_idx["tag"] == col]["context"].values
        df_num_elem.at[code, col] = len(df_idx[df_idx["tag"] == col])
        if len(context_list) != 0:
            dict_context[col].extend(context_list)

for key in dict_context.keys():
    dict_context[key] = list(set(dict_context[key]))

In [112]:
dict_context

{'EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults': ['CurrentYearInstant']}

# US GAAP

In [113]:
df_num_elem = pd.DataFrame(index = df_accounting[df_accounting["accounting"] == "US GAAP"].index, columns = col_USGAAP)
dict_context = dict(zip(col_USGAAP, [[] for _ in range(len(col_USGAAP))]))

for idx, code in enumerate(df_num_elem.index):
    if idx % 10 == 0:
        print(idx, end = "\r")
    df_idx = all_df_dic["有価証券報告書"][code]
    df_idx = df_idx[df_idx["提出日"] == max(df_idx["提出日"].unique())]
    df_idx = df_idx[df_idx["context"].str.contains("CurrentYearInstant")]
    for col in col_USGAAP:
        context_list = df_idx[df_idx["tag"] == col]["context"].values
        df_num_elem.at[code, col] = len(df_idx[df_idx["tag"] == col])
        if len(context_list) != 0:
            dict_context[col].extend(context_list)

for key in dict_context.keys():
    dict_context[key] = list(set(dict_context[key]))

In [114]:
dict_context

{'EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults': ['CurrentYearInstant']}